In [29]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv(r"G:\LILTHOMA\Rise_Wpu\Mentor_Mind\Recommender_System_Using_Python\dataset\books_data\FinalData.csv")
ratings=pd.read_csv(r"G:\LILTHOMA\Rise_Wpu\Mentor_Mind\Recommender_System_Using_Python\dataset\books_data\ratings.csv")
rating_count=pd.read_csv(r"G:\LILTHOMA\Rise_Wpu\Mentor_Mind\Recommender_System_Using_Python\dataset\books_data\Ratingscount.csv")
avg_ratings=pd.read_csv(r"G:\LILTHOMA\Rise_Wpu\Mentor_Mind\Recommender_System_Using_Python\dataset\books_data\AverageRatings.csv")

In [3]:
data.head()

,book_id,authors,title,Genres
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,Fantasy;Young-Age
2,3,Stephenie Meyer,"Twilight (Twilight, #1)",Fantasy
3,4,Harper Lee,To Kill a Mockingbird,Self-Help;Drama
4,5,F. Scott Fitzgerald,The Great Gatsby,Drama


In [4]:
ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [5]:
rating_count.head()

,book_id,rating
0,1,22806
1,2,21850
2,3,16931
3,4,19088
4,5,16604


In [6]:
avg_ratings.head()

,book_id,rating
0,1,4.279707
1,2,4.351350
2,3,3.214341
3,4,4.329369
4,5,3.772224


In [7]:
data['avg_ratings']=avg_ratings.set_index("book_id")['rating'][data["book_id"]].values
data['rating_count']=rating_count.set_index("book_id")['rating'][data['book_id']].values

In [8]:
data.head()

,book_id,authors,title,Genres,avg_ratings,rating_count
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,4.279707,22806
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,Fantasy;Young-Age,4.351350,21850
2,3,Stephenie Meyer,"Twilight (Twilight, #1)",Fantasy,3.214341,16931
3,4,Harper Lee,To Kill a Mockingbird,Self-Help;Drama,4.329369,19088
4,5,F. Scott Fitzgerald,The Great Gatsby,Drama,3.772224,16604


# Content Based

In [122]:
#Cleaning Title
import re
def clean_title(title):
    return re.sub("[^a-zA-Z0-9]"," ",title) 

In [21]:
data['clean_title']=data['title'].apply(clean_title)

In [124]:
#For a content Based Approach we will use Names
from sklearn.feature_extraction.text import TfidfVectorizer
tfv=TfidfVectorizer(ngram_range=(1,2))

In [126]:
#Importing Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

In [128]:
#Creating a sparse_matrix
title_vector=tfv.fit_transform(data['clean_title'])

In [129]:
#Function for Content Recommendation
def content_rec(entered_title):
    cleaned_title=clean_title(entered_title)
    new=tfv.transform([cleaned_title])
    similar_title=cosine_similarity(new,title_vector).flatten()
    indx=np.argpartition(similar_title,-5)[-5:]
    result = data.iloc[indx][::-1]
    return result

In [130]:
#EXapmple of usage of this function
content=content_rec("Harry Potter")
#We will use the book_id for Collaborative filtering
content.iloc[0]['book_id']


2

In [131]:
ratings

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
99892,999,46527,5
99893,999,47528,5
99894,999,47746,5
99895,999,49830,5


# Collaborative

In [132]:
#Creating Pivot_Table for finding Knn cosine similarity
pivot_table=ratings.pivot_table(index="book_id",columns="user_id",values="rating").fillna(0)
pivot_table

user_id,7,35,41,75,143,146,153,158,173,178,...,53292,53293,53318,53332,53352,53354,53356,53366,53381,53403
book_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [134]:
#Creating array_matrix 
from scipy.sparse import csr_matrix
book_features=csr_matrix(pivot_table.values)

In [135]:
#Creating Knn model using cosine similarity
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric="cosine",algorithm="brute")
model_knn.fit(book_features)

NearestNeighbors(algorithm='brute', metric='cosine')

In [137]:
# Testing of model by taking book id as 1
query_index=1

In [138]:
distances,indices=model_knn.kneighbors(pivot_table.loc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [139]:
#To get the indexes of the books most liked by users with similar taste

collaborative_indexes=[]
for i in range(0,len(distances.flatten())):
    collaborative_indexes.append(pivot_table.index[indices.flatten()[i]])
collaborative_indexes

[1, 17, 31, 2, 20, 3]

In [140]:
data[data["book_id"].isin(collaborative_indexes)]

,book_id,authors,title,Genres,avg_ratings,rating_count,clean_title
0,1,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)",SciFi;Drama,4.279707,22806,The Hunger Games The Hunger Games 1
1,2,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,Fantasy;Young-Age,4.351350,21850,Harry Potter and the Sorcerer s Stone Harry P...
2,3,Stephenie Meyer,"Twilight (Twilight, #1)",Fantasy,3.214341,16931,Twilight Twilight 1
16,17,Suzanne Collins,"Catching Fire (The Hunger Games, #2)",SciFi;Drama,4.133422,16549,Catching Fire The Hunger Games 2
19,20,Suzanne Collins,"Mockingjay (The Hunger Games, #3)",Fantasy,3.853131,15953,Mockingjay The Hunger Games 3
30,31,Kathryn Stockett,The Help,History;Fiction,4.382887,12727,The Help


In [141]:
#Final Function for Collaborative Approach
def knn_collab(book_id):
    query_index=book_id
    distances,indices=model_knn.kneighbors(pivot_table.loc[query_index,:].values.reshape(1,-1),n_neighbors=6)
    collaborative_indexes=[]
    for i in range(0,len(distances.flatten())):
        collaborative_indexes.append(pivot_table.index[indices.flatten()[i]])
    knn_result=data[data["book_id"].isin(collaborative_indexes)]
    return knn_result
    
    
    
    

# Hybrid Recommendation

In [142]:
def final_recommend(movie_title):
    content=content_rec(movie_title)
    ids=content.iloc[0]['book_id']
    collab=knn_collab(ids)
    result=pd.concat([content,collab],axis=0).drop_duplicates()
    return result[['title',"authors","Genres"]]
    

In [143]:
final_recommend("Harry Potter")

,title,authors,Genres
1,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",Fantasy;Young-Age
421,"Harry Potter Boxset (Harry Potter, #1-7)",J.K. Rowling,Fantasy;Kids
24,Harry Potter and the Deathly Hallows (Harry Po...,"J.K. Rowling, Mary GrandPré",Fantasy;Young-Age
23,Harry Potter and the Goblet of Fire (Harry Pot...,"J.K. Rowling, Mary GrandPré",Fantasy;Young-Age
22,Harry Potter and the Chamber of Secrets (Harry...,"J.K. Rowling, Mary GrandPré",Fantasy;Young-Age
3,To Kill a Mockingbird,Harper Lee,Self-Help;Drama
4,The Great Gatsby,F. Scott Fitzgerald,Drama
12,1984,"George Orwell, Erich Fromm, Celâl Üster",Thriller
18,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,Fantasy
32,Memoirs of a Geisha,Arthur Golden,Biography


In [144]:
final_recommend("Lord of the Rings")

,title,authors,Genres
154,"The Two Towers (The Lord of the Rings, #2)",J.R.R. Tolkien,Fantasy;Thriller
160,"The Return of the King (The Lord of the Rings,...",J.R.R. Tolkien,Fantasy;Thriller
188,"The Lord of the Rings (The Lord of the Rings, ...",J.R.R. Tolkien,Fantasy
18,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,Fantasy
27,Lord of the Flies,William Golding,Psychological;Young-Age
17,Harry Potter and the Prisoner of Azkaban (Harr...,"J.K. Rowling, Mary GrandPré, Rufus Beck",Fantasy;Fiction;Young-Age
218,The Chronicles of Narnia (Chronicles of Narnia...,"C.S. Lewis, Pauline Baynes",Fantasy;Kids


In [149]:
final_recommend("The Alchemist")

,title,authors,Genres
34,The Alchemist,"Paulo Coelho, Alan R. Clarke",Fiction;Fantasy
160,"The Return of the King (The Lord of the Rings,...",J.R.R. Tolkien,Fantasy;Thriller
18,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien,Fantasy
892,"Fullmetal Alchemist, Vol. 1 (Fullmetal Alchemi...","Hiromu Arakawa, Akira Watanabe",SciFi;Fantasy
611,"The Drawing of the Three (The Dark Tower, #2)",Stephen King,Fantasy;Horror
2,"Twilight (Twilight, #1)",Stephenie Meyer,Fantasy
7,The Catcher in the Rye,J.D. Salinger,Classic;Young-Age
32,Memoirs of a Geisha,Arthur Golden,Biography
46,The Book Thief,Markus Zusak,Young-Age;History;Fiction
47,Fahrenheit 451,Ray Bradbury,Fantasy;Fiction


In [152]:
final_recommend("Sherlock homes")

,title,authors,Genres
768,The Complete Sherlock Holmes,Arthur Conan Doyle,Fiction
513,The Adventures of Sherlock Holmes,Arthur Conan Doyle,Fiction;Thriller;Drama
331,"The Magician's Nephew (Chronicles of Narnia, #6)",C.S. Lewis,Kids;Fantasy
332,Still Alice,Lisa Genova,Drama
329,"The Eye of the World (Wheel of Time, #1)",Robert Jordan,Kids;Fantasy;Fiction
292,Treasure Island,Robert Louis Stevenson,Kids;Fiction;Thriller
445,A Study in Scarlet,Arthur Conan Doyle,Thriller
617,The Hound of the Baskervilles,"Arthur Conan Doyle, Anne Perry",Thriller
671,"Murder on the Orient Express (Hercule Poirot, ...",Agatha Christie,Classic;Thriller
